 ## парафраз

In [1]:
from xml.etree import ElementTree
import itertools
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize


In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from gensim.models.keyedvectors import FastTextKeyedVectors
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize

In [3]:
m = FastTextKeyedVectors.load('model.model')

Создаем датафрейм:

In [4]:
with open('paraphrases.xml', encoding='utf-8') as f:
    data = f.read()

In [5]:
root = ElementTree.XML(data)

In [6]:
par_data = {'text1': [], 'text2': [], 'class': []}
for par in root[1]:
    par_data['text1'].append(par[3].text)
    par_data['text2'].append(par[4].text)
    par_data['class'].append(int(par[6].text))

In [7]:
parphrase_df = pd.DataFrame(par_data)
parphrase_df.head(10)

,text1,text2,class
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0
5,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,1
6,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,-1
7,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,0
8,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,-1
9,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,-1


In [8]:
len(parphrase_df)

7227

Вот здесь я читала, какие есть способы измерения расстояния в склерн и чем они отличаются
https://stackoverflow.com/questions/37303146/distance-metrics-in-scikit-learn


##### как минимум 4 способами (векторизация + расстояние) посчитайте расстояние между каждой парой предложений (из text1 и text2)


1. CountVectorizer

Сначала попробую с помощью каунтвекторайзера со стандартными параметрами

In [9]:
vectorizer1 = CountVectorizer()
vectorizer1.fit(parphrase_df.text1 + parphrase_df.text2)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Посчитаю расстояние тремя способами, которые есть в склерн

In [10]:
# функция, которая берет данные из строки и считает евклидово расстояние
def euc_dist1(row):
    sent1 = vectorizer1.transform([row['text1']])
    sent2 = vectorizer1.transform([row['text2']])
    return euclidean_distances(sent1, sent2)[0][0]
    

In [11]:
# функция, которая берет данные из строки и считает косинусную близость
def cos_dist1(row):
    sent1 = vectorizer1.transform([row['text1']])
    sent2 = vectorizer1.transform([row['text2']])
    return cosine_similarity(sent1, sent2)[0][0]
    

In [12]:
# функция, которая берет данные из строки и считает манхэттенское расстояние
def man_dist1(row):
    sent1 = vectorizer1.transform([row['text1']])
    sent2 = vectorizer1.transform([row['text2']])
    return manhattan_distances(sent1, sent2)[0][0]
    

Поименую все будущие столбцы с расстояниями просто цифрами, чтобы потом было удобнее их перебирать

In [13]:
parphrase_df['1'] = parphrase_df.apply(euc_dist1, axis=1)

In [14]:
parphrase_df['2'] = parphrase_df.apply(cos_dist1, axis=1)

In [15]:
parphrase_df['3'] = parphrase_df.apply(man_dist1, axis=1)

In [16]:
parphrase_df.head()

,text1,text2,class,1,2,3
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,3.000000,0.400892,9.0
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,3.000000,0.308607,9.0
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,2.828427,0.428571,8.0
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,3.162278,0.377964,10.0
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,3.162278,0.377964,10.0


2. CountVectorizer с параметрами


про мин-дф и макс-дф я прочла здесь
https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer

взяла ngram_range=(1, 2), чтобы учесть и просто слова и биграммы

In [17]:
# возьму расширенный список стоп-слов
with open('stopwords-ru.txt', encoding='utf-8') as f:
    content = f.readlines()
stopwords_full = [x.strip() for x in content]

In [19]:
vectorizer2 = CountVectorizer(stop_words=stopwords_full, max_df = 0.90, ngram_range=(1, 2),analyzer = 'word', tokenizer = word_tokenize)
vectorizer2.fit(parphrase_df.text1 + parphrase_df.text2)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None,
        stop_words=['c', 'а', 'алло', 'без', 'белый', 'близко', 'более', 'больше', 'большой', 'будем', 'будет', 'будете', 'будешь', 'будто', 'буду', 'будут', 'будь', 'бы', 'бывает', 'бывь', 'был', 'была', 'были', 'было', 'быть', 'в', 'важная', 'важное', 'важные', 'важный', 'вам', 'вами', 'вас', 'ваш', 'ваша...и', 'этим', 'этими', 'этих', 'это', 'этого', 'этой', 'этом', 'этому', 'этот', 'эту', 'я', 'являюсь'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x0000021C1863F510>,
        vocabulary=None)

Функции, которые будут возвращать расстояние между предложениями:

In [20]:
def euc_dist2(row):
    sent1 = vectorizer2.transform([row['text1']])
    sent2 = vectorizer2.transform([row['text2']])
    return euclidean_distances(sent1, sent2)[0][0]

In [21]:
def cos_dist2(row):
    sent1 = vectorizer2.transform([row['text1']])
    sent2 = vectorizer2.transform([row['text2']])
    return cosine_similarity(sent1, sent2)[0][0]

In [22]:
def man_dist2(row):
    sent1 = vectorizer2.transform([row['text1']])
    sent2 = vectorizer2.transform([row['text2']])
    return manhattan_distances(sent1, sent2)[0][0]
    

In [23]:
parphrase_df['4'] = parphrase_df.apply(euc_dist2, axis=1)

In [24]:
parphrase_df['5'] = parphrase_df.apply(cos_dist2, axis=1)

In [25]:
parphrase_df['6'] = parphrase_df.apply(man_dist2, axis=1)

In [26]:
parphrase_df.head()

,text1,text2,class,1,2,3,4,5,6
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,3.000000,0.400892,9.0,3.741657,0.365148,14.0
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,3.000000,0.308607,9.0,4.123106,0.261116,17.0
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,2.828427,0.428571,8.0,4.123106,0.370625,17.0
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,3.162278,0.377964,10.0,4.358899,0.240192,19.0
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,3.162278,0.377964,10.0,4.898979,0.201802,24.0


3. TruncatedSVD

Делаю то же самое третьим способом:

In [27]:
svd = TruncatedSVD(n_components=300, n_iter=7)

In [28]:
vectorizer3 = CountVectorizer()
bow = vectorizer3.fit_transform(parphrase_df.text1 + parphrase_df.text2)
svd.fit(bow)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=7,
       random_state=None, tol=0.0)

In [29]:
def euc_dist3(row):
    sent1 = svd.transform(vectorizer3.transform([row['text1']]))
    sent2 = svd.transform(vectorizer3.transform([row['text2']]))
    return euclidean_distances(sent1, sent2)[0][0]

In [30]:
def cos_dist3(row):
    sent1 = svd.transform(vectorizer3.transform([row['text1']]))
    sent2 = svd.transform(vectorizer3.transform([row['text2']]))
    return cosine_similarity(sent1, sent2)[0][0]

In [31]:
def man_dist3(row):
    sent1 = svd.transform(vectorizer3.transform([row['text1']]))
    sent2 = svd.transform(vectorizer3.transform([row['text2']]))
    return manhattan_distances(sent1, sent2)[0][0]

In [32]:
parphrase_df['7'] = parphrase_df.apply(euc_dist3, axis=1)

In [33]:
parphrase_df['8'] = parphrase_df.apply(cos_dist3, axis=1)

In [34]:
parphrase_df['9'] = parphrase_df.apply(man_dist3, axis=1)

In [35]:
parphrase_df.head()

,text1,text2,class,1,2,3,4,5,6,7,8,9
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,3.000000,0.400892,9.0,3.741657,0.365148,14.0,1.350238,0.550983,13.301454
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,3.000000,0.308607,9.0,4.123106,0.261116,17.0,1.022669,0.732213,4.863660
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,2.828427,0.428571,8.0,4.123106,0.370625,17.0,1.314963,0.334648,13.643393
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,3.162278,0.377964,10.0,4.358899,0.240192,19.0,1.502144,0.732007,15.234192
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,3.162278,0.377964,10.0,4.898979,0.201802,24.0,2.033996,0.624474,24.367385


4. word2vec ( тайга )

и четвертым способом:

In [36]:
def embed_text(text):
    text = word_tokenize(text)
    vec_sum = 0
    for el in text:
        try:
            vec_sum += m[el]
        except:
            pass
    return vec_sum

In [37]:
def euc_dist4(row):
    sent1 = [embed_text(row['text1'])]
    sent2 = [embed_text(row['text2'])]
    return euclidean_distances(sent1, sent2)[0][0]

In [38]:
def cos_dist4(row):
    sent1 = [embed_text(row['text1'])]
    sent2 = [embed_text(row['text2'])]
    return cosine_similarity(sent1, sent2)[0][0]

In [39]:
def man_dist4(row):
    sent1 = [embed_text(row['text1'])]
    sent2 = [embed_text(row['text2'])]
    return manhattan_distances(sent1, sent2)[0][0]

In [40]:
parphrase_df['10'] = parphrase_df.apply(euc_dist4, axis=1)

In [41]:
parphrase_df['11'] = parphrase_df.apply(cos_dist4, axis=1)

In [42]:
parphrase_df['12'] = parphrase_df.apply(man_dist4, axis=1)

In [43]:
parphrase_df.head(10)

,text1,text2,class,1,2,3,4,5,6,7,8,9,10,11,12
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,3.000000,0.400892,9.0,3.741657,0.365148,14.0,1.350238,0.550983,13.301454,71.748291,0.788704,976.753424
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,3.000000,0.308607,9.0,4.123106,0.261116,17.0,1.022669,0.732213,4.863660,72.766319,0.601266,975.334097
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,2.828427,0.428571,8.0,4.123106,0.370625,17.0,1.314963,0.334648,13.643393,84.052299,0.607760,1123.061437
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,3.162278,0.377964,10.0,4.358899,0.240192,19.0,1.502144,0.732007,15.234192,94.067535,0.634141,1321.738400
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,3.162278,0.377964,10.0,4.898979,0.201802,24.0,2.033996,0.624474,24.367385,67.911362,0.793717,955.087932
5,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,1,2.449490,0.500000,6.0,3.605551,0.435194,13.0,0.905337,0.538458,9.759748,53.680878,0.755837,736.760647
6,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,-1,4.000000,0.000000,16.0,4.795832,0.080582,23.0,2.141541,0.088635,23.541625,115.997894,0.304224,1610.742023
7,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,0,2.449490,0.625000,6.0,3.605551,0.521749,13.0,0.917058,0.822436,11.203557,43.462032,0.913547,611.457537
8,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,-1,4.690416,0.000000,20.0,4.898979,0.077152,24.0,2.508501,0.110205,25.380437,150.463181,0.305421,2084.780885
9,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,-1,4.000000,0.000000,16.0,5.099020,0.072169,26.0,2.006057,0.100294,22.074074,128.294266,0.354124,1842.915286


5. Tf-idf Vectorizer

И попробую еще тфдф векторайзер, просто так, посмотреть, будет ли от него польза

In [117]:
vectorizer4 = TfidfVectorizer()
vectorizer4.fit(parphrase_df.text1 + parphrase_df.text2)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [118]:
def euc_dist5(row):
    sent1 = vectorizer4.transform([row['text1']])
    sent2 = vectorizer4.transform([row['text2']])
    return euclidean_distances(sent1, sent2)[0][0]
    

In [119]:
def cos_dist5(row):
    sent1 = vectorizer4.transform([row['text1']])
    sent2 = vectorizer4.transform([row['text2']])
    return cosine_similarity(sent1, sent2)[0][0]
    

In [120]:
def man_dist5(row):
    sent1 = vectorizer4.transform([row['text1']])
    sent2 = vectorizer4.transform([row['text2']])
    return manhattan_distances(sent1, sent2)[0][0]
    

In [121]:
parphrase_df['13'] = parphrase_df.apply(euc_dist5, axis=1)

In [122]:
parphrase_df['14'] = parphrase_df.apply(cos_dist5, axis=1)

In [123]:
parphrase_df['15'] = parphrase_df.apply(man_dist5, axis=1)

In [124]:
parphrase_df.head(10)

,text1,text2,class,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,3.000000,0.400892,9.0,3.741657,0.365148,14.0,1.350238,0.550983,13.301454,71.748291,0.788704,976.753424,1.075894,0.421226,3.178169
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,3.000000,0.308607,9.0,4.123106,0.261116,17.0,1.022669,0.732213,4.863660,72.766319,0.601266,975.334097,1.193534,0.287738,3.536441
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,2.828427,0.428571,8.0,4.123106,0.370625,17.0,1.314963,0.334648,13.643393,84.052299,0.607760,1123.061437,1.070642,0.426863,3.029923
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,3.162278,0.377964,10.0,4.358899,0.240192,19.0,1.502144,0.732007,15.234192,94.067535,0.634141,1321.738400,1.262361,0.203222,3.954934
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,3.162278,0.377964,10.0,4.898979,0.201802,24.0,2.033996,0.624474,24.367385,67.911362,0.793717,955.087932,1.232313,0.240702,3.761465
5,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,1,2.449490,0.500000,6.0,3.605551,0.435194,13.0,0.905337,0.538458,9.759748,53.680878,0.755837,736.760647,1.006765,0.493212,2.506694
6,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,-1,4.000000,0.000000,16.0,4.795832,0.080582,23.0,2.141541,0.088635,23.541625,115.997894,0.304224,1610.742023,1.414214,0.000000,5.440805
7,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,0,2.449490,0.625000,6.0,3.605551,0.521749,13.0,0.917058,0.822436,11.203557,43.462032,0.913547,611.457537,0.871304,0.620415,2.141933
8,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,-1,4.690416,0.000000,20.0,4.898979,0.077152,24.0,2.508501,0.110205,25.380437,150.463181,0.305421,2084.780885,1.414214,0.000000,5.793233
9,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,-1,4.000000,0.000000,16.0,5.099020,0.072169,26.0,2.006057,0.100294,22.074074,128.294266,0.354124,1842.915286,1.414214,0.000000,5.441300


In [ ]:
# запишу получившийся датафрейм в файл, чтоб всё не пересчитывать, если понадобится потом
parphrase_df.to_csv('vectors.csv', index=False, encoding='utf-8')

##### составьте из этих расстояний признаки для обучения (матрицу признаков, или список списков признаков)

In [127]:
# вытаскиваю значения из нужных столбцов в виде аррея и складываю их в переменную а
a = parphrase_df.iloc[:, 3:].copy()
a = a.values
a



array([[ 3.        ,  0.40089186,  9.        , ...,  1.0758938 ,
         0.42122627,  3.1781688 ],
       [ 3.        ,  0.3086067 ,  9.        , ...,  1.19353443,
         0.28773779,  3.53644069],
       [ 2.82842712,  0.42857143,  8.        , ...,  1.07064151,
         0.42686337,  3.02992325],
       ...,
       [ 3.16227766,  0.16666667, 10.        , ...,  1.34054463,
         0.10147004,  4.21629161],
       [ 3.60555128,  0.13608276, 13.        , ...,  1.35137867,
         0.08688785,  4.85723634],
       [ 2.64575131,  0.36514837,  7.        , ...,  1.26229655,
         0.2033037 ,  3.44741773]])

In [166]:
type(a)

numpy.ndarray

In [128]:
# 15 столбцов, строк столько же, сколько в датафрейме
a.shape

(7227, 15)

In [53]:
a[0]

array([3.00000000e+00, 4.00891863e-01, 9.00000000e+00, 3.74165739e+00,
       3.65148372e-01, 1.40000000e+01, 1.35023828e+00, 5.50982596e-01,
       1.33014542e+01, 7.17482910e+01, 7.88703918e-01, 9.76753424e+02])

##### на этих признаках обучите любой классификатор на парафраз (предсказывать столбец `class`)


In [180]:
# эта штука будет отлавливать предупреждения
import warnings
warnings.filterwarnings('ignore')

In [168]:
X_train, X_test, y_train, y_test = train_test_split(a, parphrase_df['class'])

In [169]:
lr = LogisticRegression(random_state=50)
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))


             precision    recall  f1-score   support

         -1       0.68      0.72      0.70       615
          0       0.55      0.69      0.61       761
          1       0.61      0.28      0.38       431

avg / total       0.61      0.60      0.59      1807



У логистической регрессии получился хороший результат. Попробую другой классификатор

In [181]:
sgd = SGDClassifier()
sgd_m = sgd.fit(X_train, y_train)
print(classification_report(y_test, sgd_m.predict(X_test)))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00       615
          0       0.42      1.00      0.59       761
          1       1.00      0.00      0.01       431

avg / total       0.42      0.42      0.25      1807



У СГД классификатора плохой результат, попробую еще решающие деревья

In [182]:
tree = DecisionTreeClassifier()
tree_m = tree.fit(X_train, y_train)
print(classification_report(y_test, tree_m.predict(X_test)))

             precision    recall  f1-score   support

         -1       0.63      0.66      0.64       615
          0       0.49      0.47      0.48       761
          1       0.36      0.35      0.36       431

avg / total       0.50      0.51      0.51      1807



Результат хуже, чем у регрессии, так что дальше буду работать с регрессией

Я хочу узнать, какие из 15 способов подсчета расстояния оказались наиболее полезными, т.е. какие из них дают лучшие результаты. 
Проверю это простым перебором

In [183]:
# Вытащу названия колонок
parphrase_df.columns.tolist()

['text1',
 'text2',
 'class',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15']

In [184]:
# присвою нужные имена переменной 
names = ['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15']

In [207]:
# делаю список всех возможных комбинаций столбцов, количество столбцов меняю для каждой проверки
combinations = list(itertools.combinations(names, 3))


Буду проверять все комбинации столбцов, насколько хорошие результаты они дают

In [189]:
# начну с проверки, будут ли хорошие результаты, если в классификатор засунуть только один столбец
for comb in combinations:
    b = parphrase_df.filter(items = comb).values
    X_train, X_test, y_train, y_test = train_test_split(b, parphrase_df['class'])
    lr = LogisticRegression(random_state=50)
    clf = lr.fit(X_train, y_train)
    if precision_recall_fscore_support(y_test, clf.predict(X_test), average='weighted')[0] > 0.60:
        print(comb)

('15',)


In [200]:
b = parphrase_df.filter(items = ['15']).values
X_train, X_test, y_train, y_test = train_test_split(b, parphrase_df['class'])
lr = LogisticRegression(random_state=50)
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

             precision    recall  f1-score   support

         -1       0.66      0.74      0.69       649
          0       0.52      0.66      0.58       729
          1       0.66      0.23      0.34       429

avg / total       0.60      0.59      0.56      1807



Столбец № 15, т.е. тф-дф векторайзер + манхэттенское расстояние само по себе дают хороший результат.

In [202]:
# с двумя столбцами
for comb in combinations:
    b = parphrase_df.filter(items = comb).values
    X_train, X_test, y_train, y_test = train_test_split(b, parphrase_df['class'])
    lr = LogisticRegression(random_state=50)
    clf = lr.fit(X_train, y_train)
    if precision_recall_fscore_support(y_test, clf.predict(X_test), average='weighted')[0] > 0.62:
        print(comb, precision_recall_fscore_support(y_test, clf.predict(X_test), average='weighted') )
        

('10', '15') (0.6319712049144347, 0.6081903707802988, 0.5935968125886211, None)


In [206]:
b = parphrase_df.filter(items = ['10', '15']).values
X_train, X_test, y_train, y_test = train_test_split(b, parphrase_df['class'])
lr = LogisticRegression(random_state=50)
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

             precision    recall  f1-score   support

         -1       0.64      0.72      0.68       644
          0       0.54      0.64      0.59       757
          1       0.65      0.30      0.41       406

avg / total       0.60      0.59      0.58      1807



Столбец 10 - это евклидово расстояние на эмбеддингах из тайги. То есть сочетание (тф-дф векторайзер + манхэттенское расстояние) и (евклидово расстояние на эмбеддингах из тайги) Тоже даёт хороший результат. 

In [209]:
# с тремя столбцами
for comb in combinations:
    b = parphrase_df.filter(items = comb).values
    X_train, X_test, y_train, y_test = train_test_split(b, parphrase_df['class'])
    lr = LogisticRegression(random_state=50)
    clf = lr.fit(X_train, y_train)
    if precision_recall_fscore_support(y_test, clf.predict(X_test), average='weighted')[0] > 0.63:
        print(comb, precision_recall_fscore_support(y_test, clf.predict(X_test), average='weighted') )

('3', '7', '10') (0.6330425601025835, 0.5655783065855008, 0.5366113799215355, None)
('3', '10', '15') (0.6363243001120419, 0.6131710016602103, 0.5950793640005654, None)


Опять среди хороших результатов столбцы "10" и "15", и еще добавился столбец "3" - это (каунт векторайзер со стандартными параметрами) + (манхэттенское расстояние)

In [212]:
b = parphrase_df.filter(items = ['3', '10', '15']).values
X_train, X_test, y_train, y_test = train_test_split(b, parphrase_df['class'])
lr = LogisticRegression(random_state=50)
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

             precision    recall  f1-score   support

         -1       0.64      0.77      0.70       628
          0       0.58      0.61      0.59       789
          1       0.61      0.35      0.44       390

avg / total       0.61      0.61      0.60      1807



Если честно, я проверяла перебором остальные сочетания столбцов тоже, но не вижу смысла сюда это включать, результаты не улучшаются, всё крутится вокруг 60%

Сейчас я хочу попробовать функции нормализации, которые есть в ск-лерне. Я прочла о них вот здесь 

https://stackoverflow.com/questions/30918781/right-function-for-normalizing-input-of-sklearn-svm

In [213]:
normalized_a = normalize(a)
normalized_a[0]

array([3.06236228e-03, 4.09225373e-04, 9.18708685e-03, 3.81943682e-03,
       3.72738867e-04, 1.42910240e-02, 1.37830626e-03, 5.62436106e-04,
       1.35779572e-02, 7.32397534e-02, 8.05099044e-04, 9.97057616e-01,
       1.09825886e-03, 4.29982480e-04, 3.24423476e-03])

In [214]:
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(normalized_a, parphrase_df['class'])

In [215]:
lr = LogisticRegression(random_state=50)
clf = lr.fit(X_train_n, y_train_n)
print(classification_report(y_test_n, clf.predict(X_test_n)))


             precision    recall  f1-score   support

         -1       0.00      0.00      0.00       648
          0       0.41      1.00      0.58       736
          1       1.00      0.02      0.04       423

avg / total       0.40      0.41      0.25      1807



Результат сильно хуже. Попробую вторую функцию

In [219]:
standardized_a = scale(a)
standardized_a[0]

array([ 0.29608664, -0.09149115,  0.179114  , -0.31848285,  0.06604557,
       -0.4212638 ,  0.00688847, -0.0911013 , -0.0564469 , -0.21314554,
        0.38228875, -0.24501991,  0.08013052,  0.0559334 ,  0.0419505 ])

In [217]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(standardized_a, parphrase_df['class'])

In [218]:
lr = LogisticRegression(random_state=50)
clf = lr.fit(X_train_s, y_train_s)
print(classification_report(y_test_s, clf.predict(X_test_s)))


             precision    recall  f1-score   support

         -1       0.72      0.69      0.70       672
          0       0.51      0.70      0.59       716
          1       0.59      0.27      0.37       419

avg / total       0.61      0.59      0.58      1807



Результат такой же, как и с обычными векторами, нестандартизированными. 